In [1]:
import requests, folium
import pandas as pd
from tqdm import tqdm
from urllib.parse import quote
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.ggpolice.go.kr/main/html.do?menu=HC44'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [3]:
lis = soup.select('li.road')
len(lis)

31

- Step 3

In [4]:
li = lis[0]
name = li.find('a').string.strip()
href = li.find('a')['href']
name, href

('수원중부서', 'https://www.ggpolice.go.kr/swjb/html.do?menu=HC24')

In [5]:
sub_res = requests.get(href)
sub_soup = BeautifulSoup(sub_res.text, 'html.parser')

In [6]:
sub_lis = sub_soup.select('.basic_list7 > li')
len(sub_lis)

3

In [10]:
name = sub_lis[0].get_text().strip()[6:]
addr = sub_lis[1].get_text().strip()[9:-9]
name, addr

('수원중부경찰서', '경기도 수원시 장안구 정자천로 199')

- Step 4

In [9]:
# 평소대로 돌려보기- 에러
lines = []
for li in lis:
    href = li.find('a')['href']
    sub_res = requests.get(href)
    sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
    sub_lis = sub_soup.select('.basic_list7 > li')
    name = sub_lis[0].get_text().strip()[6:]
    addr = sub_lis[1].get_text().strip()[9:-9]
    lines.append({'경찰서':name, '주소':addr})

IndexError: list index out of range

- step 5. 오류해결

In [10]:
# 에러가 발생하는지 파악
lines = []
for index, li in enumerate(lis):    # for문에 추가 
    try:
        href = li.find('a')['href']
        sub_res = requests.get(href)
        sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
        sub_lis = sub_soup.select('.basic_list7 > li')
        name = sub_lis[0].get_text().strip()[6:]
        addr = sub_lis[1].get_text().strip()[9:-9]
        lines.append({'name':name, 'addr':addr})
    except:
        print(index)
#===> 문제의 원인(인덱스값) 출력

6
25


In [11]:
# 에러를 찾아서 해결
# index 6(군포경찰서)을 찾아서 확인
li = lis[6]
href = li.find('a')['href']
print(href)
sub_res = requests.get(href)
sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
sub_lis = sub_soup.select('.basic_list7 > li')
print(len(sub_lis))
# 정상: 3이 나와야함/ 비정상

https://www.ggpolice.go.kr/gunpo/html.do?menu=HC24
0


In [12]:
# 
lines = []
for li in lis:
    href = li.find('a')['href']
    sub_res = requests.get(href)
    sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
    sub_lis = sub_soup.select('.basic_list7 > li')
    if len(sub_lis) == 0:
        continue   # 0이면 밑의 작업 x 위로 돌아감

    name = sub_lis[0].get_text().strip()[6:]
    addr = sub_lis[1].get_text().strip()[9:-9]
    lines.append({'경찰서': name, '주소':addr})

In [13]:
# 정확한 원인을 찾지 못한 경우- just skip
lines = []
for li in lis:
    try:
        href = li.find('a')['href']
        sub_res = requests.get(href)
        sub_soup = BeautifulSoup(res.text, 'html.parser')
        sub_li = sub_soup.select('.basic_list7 > li')
        name = sub_lis[0].get_text().strip()[6:]
        addr = sub_lis[1].get_text().strip()[9:-9]
        lines.append({'경찰서': name, '주소':addr})
    except:
        continue

- DF만들기

In [15]:
df = pd.DataFrame(lines)
df.head()

,경찰서,주소
0,하남경찰서,경기 하남시 검단로 27
1,하남경찰서,경기 하남시 검단로 27
2,하남경찰서,경기 하남시 검단로 27
3,하남경찰서,경기 하남시 검단로 27
4,하남경찰서,경기 하남시 검단로 27


In [18]:
# 중복여부 확인
len(df.경찰서.unique())

1

```drop duplicate```

In [17]:
# 중복제거
df.drop_duplicates(subset='name', inplace = True)
df.shape

KeyError: Index(['name'], dtype='object')

- 위도경도 불러오기

- 지도에 표시하기